In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pylab import hist, show
import scipy
# import zipfile
import ipystata


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

IPyStata is loaded in batch mode.


/home/kevin/miniconda3/envs/idp/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


#### $Win = \beta_{0} + \beta_{1}MeanC + \beta_{2}MeanW + \beta_{3}MeanD + \beta_{4}MeanG + e_{s}$

- A balanced roster will have one player ranked in each centre position (C1, C2, C3, C4), two wingers ranked on every line (LW1, RW1, LW2,RW2, etc) and two defensemen ranked in all three defensive pairings. 
- The ranking of a balanced roster is 2.5 [(1+2+3+4)/4] centres, 2.5 [(1+1+2+2+3+3+4+4)/8] for wingers and 2 [(1+1+2+2+3+3)/6] for defensemen.

- Since players are ranked from 1 to 4 for forwards and 1 to 3 for defensemen, 1 being the highest ranked, a team is considered to have an above average roster when the **mean of each forward position is smaller than 2.5 and the mean of defensive pairings is smaller than 2**. 

### games with 4 centers, 8 wingers, 6 defensemen and 1 goaltender

In [134]:
# da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2010_games_with_4c_8w_6d_1g.csv')

#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2010_games_with_4c_8w_6d_1g.csv')
# da = pd.read_csv('/mnt/intel1200/projects/git/_data/2010_2017_games_with_12f_6d.csv')

da = pd.read_csv('/mnt/intel1200/projects/git/_data/2010_2017_games_with_4c_8w_6d_1g.csv')
da = da.drop('Unnamed: 0', axis=1)
da = da.sort_values(by=['Season', 'GameNumber', 'TeamCode'])
da['GD'] = da['GF'] - da['GA']
da.head()

,Season,GameNumber,WinTeam,LossTeam,GF,GA,GD,TeamCode,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG,Sum,meanc,meanw,meand,meang,DC1,DC2,DC3,DW1,DW2,DW3,DD1,DD2,DG1,DG2
0,2010,20023,LA,VAN,1,1,0,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,1,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
1,2010,20023,LA,VAN,1,1,0,VAN,19.0,4.0,8.0,6.0,1.0,2.0,0.0,1.0,1.0,3.0,2.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,2.0,0,2.25,2.250,1.666667,1.0,7.166667,0.25,0.250,0.333333,1.0,1.0,-1.0,0.0,1.0,0.0,-1.0,2.0,1.0,1.0,0.0
3,2010,20025,CGY,LA,3,1,2,CGY,19.0,4.0,8.0,6.0,1.0,2.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,2.0,4.0,0.0,2.0,1,2.00,2.250,2.166667,2.0,8.416667,0.50,0.250,-0.166667,0.0,1.0,0.0,-1.0,0.0,2.0,-2.0,-1.0,1.0,0.0,1.0
2,2010,20025,CGY,LA,1,3,-2,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,0,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
4,2010,20037,WSH,NYI,1,2,-1,NYI,19.0,4.0,8.0,6.0,1.0,1.0,2.0,0.0,1.0,0.0,4.0,2.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,0,2.25,2.625,2.333333,2.0,9.208333,0.25,-0.125,-0.333333,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,-2.0,2.0,0.0,1.0


In [135]:
df = da.groupby(['Win'])['MeanC', 'MeanW', 'MeanD', 'MeanG'].mean()
df = df.T
df['bf'] = df[1]/df[0]
df

Win,0,1,bf
MeanC,2.157209,2.059049,0.954497
MeanW,2.167561,2.082439,0.960729
MeanD,1.941718,1.842025,0.948657
MeanG,1.920245,1.785276,0.929712


In [136]:
ds = da.groupby(['Season', 'TeamCode'])[['TeamCode', 'MeanC', 'MeanW', 'MeanD']].mean()
ds.xs(2017, level='Season')

,MeanC,MeanW,MeanD
TeamCode,,,
ARI,2.708333,2.208333,1.694444
CAR,2.500000,2.031250,1.791667
CBJ,3.333333,1.708333,1.555556
CGY,2.625000,2.500000,2.083333
COL,1.750000,2.125000,2.166667
EDM,1.250000,2.312500,2.250000
FLA,1.750000,2.500000,1.833333
MIN,1.788462,1.884615,1.512821
N.J,2.150000,1.875000,1.600000


In [137]:
dc = da.copy ()
dc['meanc'] = 2.5 - dc['MeanC']
dc['meanw'] = 2.5 - dc['MeanW']
dc['meand'] = 2 - dc['MeanD']
dc['meang'] = 2 - dc['MeanG']


In [138]:
ds = dc.groupby(['Season', 'TeamCode'])[['TeamCode', 'meanc', 'meanw', 'meand']].mean()
ds.xs(2017, level='Season')

,meanc,meanw,meand
TeamCode,,,
ARI,-0.208333,0.291667,0.305556
CAR,0.000000,0.468750,0.208333
CBJ,-0.833333,0.791667,0.444444
CGY,-0.125000,0.000000,-0.083333
COL,0.750000,0.375000,-0.166667
EDM,1.250000,0.187500,-0.250000
FLA,0.750000,0.000000,0.166667
MIN,0.711538,0.615385,0.487179
N.J,0.350000,0.625000,0.400000


In [139]:
dc[['meanc', 'meanw', 'meand', 'Win', 'GD']].describe()

,meanc,meanw,meand,Win,GD
count,652.000000,652.000000,652.000000,652.000000,652.000000
mean,0.391871,0.375000,0.108129,0.500000,0.000000
std,0.454330,0.346213,0.319781,0.500384,2.227464
min,-1.000000,-0.625000,-0.666667,0.000000,-7.000000
25%,0.000000,0.125000,-0.166667,0.000000,-2.000000
50%,0.500000,0.375000,0.166667,0.500000,0.000000
75%,0.750000,0.625000,0.333333,1.000000,2.000000
max,1.500000,1.500000,0.666667,1.000000,7.000000


In [140]:
y = dc['Win'] 
y2 = dc['GA']
X2 = sm.add_constant(dc[['meanc', 'meanw', 'meand']] )
m2 = sm.OLS(y, X2).fit()
m3 = sm.OLS(y2, X2).fit()
m2.summary()
# m3.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     7.281
Date:                Fri, 11 May 2018   Prob (F-statistic):           8.29e-05
Time:                        17:26:36   Log-Likelihood:                -462.41
No. Observations:                 652   AIC:                             932.8
Df Residuals:                     648   BIC:                             950.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4175      0.031     13.310      0.000       0.356       0.479
meanc          0.0602      0.045      1.325      0.186      -0.029       0.149
meanw          0.1059      0.060      1.775      0.076      -0.011       0.223
meand          0.1774      0.067      2.653      0.008       0.046       0.309
==============================================================================
Omnibus:                        0.002   Durbin-Watson:                   3.011
Prob(Omnibus):                  0.999   Jarque-Bera (JB):               94.808
Skew:                           0.004   Prob(JB):                     2.59e-21
Kurtosis:                       1.132   Cond. No.                         4.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [141]:
dc['m2hat'] = m2.predict()
ds = dc.groupby(['Season', 'TeamCode'])[['TeamCode', 'Win', 'm2hat']].mean()
ds = ds.reset_index()
ds['pdiff'] = ds.Win - ds.m2hat
ds = ds.sort_values(by=['Season', 'Win'], ascending=[0, 0])
ds = ds[ds.Season==2017]
ds
# ds.xs(2017, level='Season')

,Season,TeamCode,Win,m2hat,pdiff
135,2017,PIT,1.000000,0.546455,0.453545
122,2017,ARI,0.833333,0.490066,0.343268
136,2017,STL,0.750000,0.482938,0.267062
124,2017,CBJ,0.666667,0.530048,0.136618
129,2017,MIN,0.615385,0.611940,0.003445
130,2017,N.J,0.600000,0.575734,0.024266
139,2017,WSH,0.600000,0.517625,0.082375
127,2017,EDM,0.500000,0.468264,0.031736
132,2017,NYI,0.500000,0.472015,0.027985
137,2017,VGK,0.500000,0.609747,-0.109747


In [142]:
dt = dc[['Season', 'GameNumber', 'TeamCode', 'C1', 'C2', 'C3', 'C4']]
dt['TC'] = dt.C1 + dt.C2 + dt.C3 + dt.C4
dt.head()

/home/kevin/miniconda3/envs/idp/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Season,GameNumber,TeamCode,C1,C2,C3,C4,TC
0,2010,20023,LA,1.0,1.0,1.0,1.0,4.0
1,2010,20023,VAN,2.0,0.0,1.0,1.0,4.0
3,2010,20025,CGY,2.0,1.0,0.0,1.0,4.0
2,2010,20025,LA,1.0,1.0,1.0,1.0,4.0
4,2010,20037,NYI,1.0,2.0,0.0,1.0,4.0


In [143]:
%%stata --data dc

gen c = meanc
gen w = meanw*2
gen d = meand*2

encode TeamCode, gen(TX)

global X "c w d"

reg Win $X
reg GF $X 
reg GA $X 
reg GD $X 


      Source |       SS           df       MS      Number of obs   =       652
-------------+----------------------------------   F(3, 648)       =      7.28
       Model |  5.31521065         3  1.77173688   Prob > F        =    0.0001
    Residual |  157.684789       648  .243340724   R-squared       =    0.0326
-------------+----------------------------------   Adj R-squared   =    0.0281
       Total |         163       651  .250384025   Root MSE        =     .4933

------------------------------------------------------------------------------
         Win |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           c |   .0601895   .0454289     1.32   0.186    -.0290162    .1493952
           w |   .0529651   .0298384     1.78   0.076    -.0056264    .1115567
           d |   .0886891   .0334323     2.65   0.008     .0230404    .1543377
       _cons |   .4175099    .031367    13.31   0.

In [147]:
%%stata --data dc

gen c = meanc
gen w = meanw*2
gen d = meand*2

encode (TeamCode), gen(TX)


gen c0 =  (C1==0)
gen c1 =  (C1==1)
gen c2 =  (C1==2)
gen c3 =  (C1==3)
gen c4 =  (C1==4)

gen lnPC1 = ln( (C1/4)*100 + 1 )
gen lnPW1 = ln( (W1/8)*100 + 1 )
gen lnPD1 = ln( (D1/6)*100 + 1 )

sum  lnPC1 lnPW1

reg Win lnPC1 lnPW1 lnPD1 
reg GF lnPC1 lnPW1 lnPD1 
reg GA lnPC1 lnPW1 lnPD1 
reg GD lnPC1 lnPW1 lnPD1 



    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       lnPC1 |        652     2.64273    1.651976          0    4.61512
       lnPW1 |        652    3.170526    .9908937          0    4.61512

      Source |       SS           df       MS      Number of obs   =       652
-------------+----------------------------------   F(3, 648)       =      5.24
       Model |  3.86396995         3  1.28798998   Prob > F        =    0.0014
    Residual |   159.13603       648  .245580293   R-squared       =    0.0237
-------------+----------------------------------   Adj R-squared   =    0.0192
       Total |         163       651  .250384025   Root MSE        =    .49556

------------------------------------------------------------------------------
         Win |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+---------------------------------------------------------------

In [ ]:
%%stata --data dc

gen c = meanc
gen w = meanw*2
gen d = meand*2

gen c0 =  (C1==0)
gen c1 =  (C1==1)
gen c2 =  (C1==2)
gen c3 =  (C1==3)
gen c4 =  (C1==4)

sum c0 c1 c2 c3 c4

tab C1

reg Win c0 c1 c2 c3

In [44]:
%%stata --data dc

gen c = meanc
gen w = meanw*2
gen d = meand*2


global X "c w d"

quietly reg Win $X
estimates store m1

quietly reg GF $X
estimates store m2

quietly reg GA $X
estimates store m3

quietly reg GD $X
estimates store m4

estimates table m1 m2 m3 m4



------------------------------------------------------------------
    Variable |     m1           m2           m3           m4      
-------------+----------------------------------------------------
           c |  .06018951    .20058891   -.03292351   -.33335373  
           w |  .05296514     .2046579   -.16984614    .23779493  
           d |   .0886891    .08797812   -.33477458    -.0046673  
       _cons |  .41750991    2.4835382    2.9473465    .20789608  
------------------------------------------------------------------


In [55]:
%%stata --data dc

encode TeamCode, gen(TX)

*reg GF C1 C2 C3 W1 W2 W3 D1 D2

reg GF C1 C2 C3 W1 W2 W3 D1 D2 bn.TX


. reg GF C1 C2 C3 W1 W2 W3 D1 D2 bn.TX
note: 34.TX omitted because of collinearity

      Source |       SS           df       MS      Number of obs   =       652
-------------+----------------------------------   F(41, 610)      =      1.30
       Model |  129.381123        41  3.15563715   Prob > F        =    0.1034
    Residual |   1479.7155       610  2.42576312   R-squared       =    0.0804
-------------+----------------------------------   Adj R-squared   =    0.0186
       Total |  1609.09663       651  2.47173061   Root MSE        =    1.5575

------------------------------------------------------------------------------
          GF |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
          C1 |   .0698964   .1426406     0.49   0.624    -.2102298    .3500226
          C2 |  -.0185654   .1266877    -0.15   0.884    -.2673625    .2302316
          C3 |  -.1007847   .1314883    -0.77

In [126]:
y = dc['Win'] 
X2 = sm.add_constant(dc[['C1', 'C2', 'C3', 'C4']] )
m2 = sm.OLS(y, X2).fit()
m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     3.521
Date:                Fri, 11 May 2018   Prob (F-statistic):             0.0149
Time:                        15:46:23   Log-Likelihood:                -467.94
No. Observations:                 652   AIC:                             943.9
Df Residuals:                     648   BIC:                             961.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0945      0.006     15.805      0.000       0.083       0.106
C1             0.1411      0.016      9.083      0.000       0.111       0.172
C2             0.1014      0.013      7.987      0.000       0.076       0.126
C3             0.0536      0.018      2.919      0.004       0.018       0.090
C4             0.0821      0.029      2.794      0.005       0.024       0.140
==============================================================================
Omnibus:                        0.001   Durbin-Watson:                   3.051
Prob(Omnibus):                  1.000   Jarque-Bera (JB):              101.738
Skew:                          -0.002   Prob(JB):                     8.09e-23
Kurtosis:                       1.065   Cond. No.                     8.92e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.89e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
y = dc['Win'] 

X1 = sm.add_constant(dc[['MeanC', 'MeanW', 'MeanD', 'MeanG']] )
X2 = sm.add_constant(dc[['meanc', 'meanw', 'meand', 'meang']] )

X3 = dc[['meanc', 'meanw', 'meand', 'meang']]
X4 = dc[['MeanC', 'MeanW', 'MeanD', 'MeanG']]

m1 = sm.OLS(y, X1).fit()
m2 = sm.OLS(y, X2).fit()
m3 = sm.OLS(y, X3).fit()
m4 = sm.OLS(y, X4).fit()

In [13]:
# db = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2011_2017_games_with_4c_8w_6d_1g.csv')
# #db = pd.readcsv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2011_2017_games_with_4c_8w_6d_1g.csv')
# db = db.drop('Unnamed: 0', axis=1)

In [14]:
da.shape

(652, 47)

In [15]:
# db.shape

### join 2010 season with 2011-2017 seasons 

In [16]:
# dataframes = [da, db]
# dc = pd.concat(dataframes)
# dc.head()
dc = da.copy()
dc.head()

,Season,GameNumber,WinTeam,LossTeam,GF,GA,GD,TeamCode,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG,Sum,meanc,meanw,meand,meang,DC1,DC2,DC3,DW1,DW2,DW3,DD1,DD2,DG1,DG2
0,2010,20023,LA,VAN,1,1,0,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,1,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
1,2010,20023,LA,VAN,1,1,0,VAN,19.0,4.0,8.0,6.0,1.0,2.0,0.0,1.0,1.0,3.0,2.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,2.0,0,2.25,2.250,1.666667,1.0,7.166667,0.25,0.250,0.333333,1.0,1.0,-1.0,0.0,1.0,0.0,-1.0,2.0,1.0,1.0,0.0
2,2010,20025,CGY,LA,1,3,2,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,0,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
3,2010,20025,CGY,LA,3,1,2,CGY,19.0,4.0,8.0,6.0,1.0,2.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,2.0,4.0,0.0,2.0,1,2.00,2.250,2.166667,2.0,8.416667,0.50,0.250,-0.166667,0.0,1.0,0.0,-1.0,0.0,2.0,-2.0,-1.0,1.0,0.0,1.0
4,2010,20037,WSH,NYI,1,2,1,NYI,19.0,4.0,8.0,6.0,1.0,1.0,2.0,0.0,1.0,0.0,4.0,2.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,0,2.25,2.625,2.333333,2.0,9.208333,0.25,-0.125,-0.333333,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,-2.0,2.0,0.0,1.0


In [17]:
dc.shape

(652, 47)

In [18]:
# dc.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2010_2017_games_with_4c_8w_6d_1g.csv', index='False', sep=',')
#dc.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/22010_2017_games_with_4c_8w_6d_1g.csv', index='False', sep=',')

In [21]:
dc = dc.sort_values(by=['Season', 'GameNumber', 'TeamCode'])
dc.head()

,Season,GameNumber,WinTeam,LossTeam,GF,GA,GD,TeamCode,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG,Sum,meanc,meanw,meand,meang,DC1,DC2,DC3,DW1,DW2,DW3,DD1,DD2,DG1,DG2
0,2010,20023,LA,VAN,1,1,0,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,1,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
1,2010,20023,LA,VAN,1,1,0,VAN,19.0,4.0,8.0,6.0,1.0,2.0,0.0,1.0,1.0,3.0,2.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,2.0,0,2.25,2.250,1.666667,1.0,7.166667,0.25,0.250,0.333333,1.0,1.0,-1.0,0.0,1.0,0.0,-1.0,2.0,1.0,1.0,0.0
3,2010,20025,CGY,LA,3,1,2,CGY,19.0,4.0,8.0,6.0,1.0,2.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,2.0,4.0,0.0,2.0,1,2.00,2.250,2.166667,2.0,8.416667,0.50,0.250,-0.166667,0.0,1.0,0.0,-1.0,0.0,2.0,-2.0,-1.0,1.0,0.0,1.0
2,2010,20025,CGY,LA,1,3,2,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,0,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0
4,2010,20037,WSH,NYI,1,2,1,NYI,19.0,4.0,8.0,6.0,1.0,1.0,2.0,0.0,1.0,0.0,4.0,2.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,0,2.25,2.625,2.333333,2.0,9.208333,0.25,-0.125,-0.333333,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,-2.0,2.0,0.0,1.0


In [26]:
dc['SumF'] =  dc.MeanC + dc.MeanW + dc.MeanD
dc.head()

,Season,GameNumber,WinTeam,LossTeam,GF,GA,GD,TeamCode,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG,Sum,meanc,meanw,meand,meang,DC1,DC2,DC3,DW1,DW2,DW3,DD1,DD2,DG1,DG2,SumF
0,2010,20023,LA,VAN,1,1,0,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,1,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0,6.708333
1,2010,20023,LA,VAN,1,1,0,VAN,19.0,4.0,8.0,6.0,1.0,2.0,0.0,1.0,1.0,3.0,2.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,2.0,0,2.25,2.250,1.666667,1.0,7.166667,0.25,0.250,0.333333,1.0,1.0,-1.0,0.0,1.0,0.0,-1.0,2.0,1.0,1.0,0.0,6.166667
3,2010,20025,CGY,LA,3,1,2,CGY,19.0,4.0,8.0,6.0,1.0,2.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,2.0,4.0,0.0,2.0,1,2.00,2.250,2.166667,2.0,8.416667,0.50,0.250,-0.166667,0.0,1.0,0.0,-1.0,0.0,2.0,-2.0,-1.0,1.0,0.0,1.0,6.416667
2,2010,20025,CGY,LA,1,3,2,LA,19.0,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0,0,2.50,2.375,1.833333,2.0,8.708333,0.00,0.125,0.166667,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,1.0,6.708333
4,2010,20037,WSH,NYI,1,2,1,NYI,19.0,4.0,8.0,6.0,1.0,1.0,2.0,0.0,1.0,0.0,4.0,2.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,0,2.25,2.625,2.333333,2.0,9.208333,0.25,-0.125,-0.333333,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,-2.0,2.0,0.0,1.0,7.208333


- summary analysis

In [44]:
df = dc.groupby(['Win'])['MeanC', 'MeanW', 'MeanD', 'MeanG'].mean()
df = df.T
df.head()

Win,0,1
MeanC,2.157209,2.059049
MeanW,2.167561,2.082439
MeanD,1.941718,1.842025
MeanG,1.920245,1.785276


In [53]:
dt = dc[dc['Season']==201]
# dx = d7[d7['TeamCode'] == 'ARI']
# dy = d7[d7['TeamCode'] == 'T.B']
# dx.head()
dt.TeamCode.value_counts()

MIN    13
WSH    10
NYI     8
ARI     6
VGK     6
N.J     5
STL     4
CAR     4
EDM     4
NYR     3
OTT     3
CBJ     3
CGY     2
NSH     1
WPG     1
COL     1
PIT     1
FLA     1
Name: TeamCode, dtype: int64

In [46]:
df['bf'] = df[1]/df[0]
df

Win,0,1,bf
MeanC,2.157209,2.059049,0.954497
MeanW,2.167561,2.082439,0.960729
MeanD,1.941718,1.842025,0.948657
MeanG,1.920245,1.785276,0.929712


### model estimation

- regress **team win percent** on the mean of players by position for games with 4 centers, 8 wingers and 6 defensemen.

In [17]:
dc['meanc'] = 2.5 - dc['MeanC']
dc['meanw'] = 2.5 - dc['MeanW']
dc['meand'] = 2 - dc['MeanD']
dc['meang'] = 2 - dc['MeanG']

dc['DC1'] = dc['C1'] - dc['C4']
dc['DC2'] = dc['C2'] - dc['C4']
dc['DC3'] = dc['C3'] - dc['C4']

dc['DW1'] = dc['W1'] - dc['W4']
dc['DW2'] = dc['W2'] - dc['W4']
dc['DW3'] = dc['W3'] - dc['W4']

dc['DD1'] = dc['D1'] - dc['D3']
dc['DD2'] = dc['D2'] - dc['D3']

dc['DG1'] = dc['G1'] - dc['G3']
dc['DG2'] = dc['G2'] - dc['G3']

In [18]:
y = dc['Win'] 

X1 = sm.add_constant(dc[['MeanC', 'MeanW', 'MeanD', 'MeanG']] )
X2 = sm.add_constant(dc[['meanc', 'meanw', 'meand', 'meang']] )

X3 = dc[['meanc', 'meanw', 'meand', 'meang']]
X4 = dc[['MeanC', 'MeanW', 'MeanD', 'MeanG']]

m1 = sm.OLS(y, X1).fit()
m2 = sm.OLS(y, X2).fit()
m3 = sm.OLS(y, X3).fit()
m4 = sm.OLS(y, X4).fit()

#m1.summary()
#m2.summary()
m3.summary()
#m4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     101.7
Date:                Thu, 10 May 2018   Prob (F-statistic):           3.37e-67
Time:                        18:36:07   Log-Likelihood:                -540.31
No. Observations:                 652   AIC:                             1089.
Df Residuals:                     648   BIC:                             1107.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
meanc          0.3071      0.047      6.575      0.000         0.215     0.399
meanw          0.5598      0.055     10.172      0.000         0.452     0.668
meand          0.0449      0.075      0.595      0.552        -0.103     0.193
meang          0.0442      0.033      1.331      0.184        -0.021     0.109
==============================================================================
Omnibus:                      181.127   Durbin-Watson:                   2.496
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.449
Skew:                           0.028   Prob(JB):                     2.44e-07
Kurtosis:                       1.943   Cond. No.                         2.91
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
X5 = dc[['DC1', 'DC2', 'DC3', 'DW1', 'DW2', 'DW3', 'DD1', 'DD2', 'DG1', 'DG2']]
X6 =  sm.add_constant(dc[['DC1', 'DC2', 'DC3', 'DW1', 'DW2', 'DW3', 'DD1', 'DD2', 'DG1', 'DG2']])

m5 = sm.OLS(y, X5).fit()
m6 = sm.OLS(y, X6).fit()

m5.summary()
#m6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     49.08
Date:                Thu, 10 May 2018   Prob (F-statistic):           1.13e-72
Time:                        18:36:25   Log-Likelihood:                -514.07
No. Observations:                 652   AIC:                             1048.
Df Residuals:                     642   BIC:                             1093.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
DC1            0.0689      0.019      3.584      0.000         0.031     0.107
DC2            0.0646      0.016      4.116      0.000         0.034     0.095
DC3           -0.0073      0.020     -0.371      0.711        -0.046     0.031
DW1            0.0625      0.012      5.285      0.000         0.039     0.086
DW2            0.0718      0.011      6.414      0.000         0.050     0.094
DW3           -0.0028      0.015     -0.186      0.852        -0.033     0.027
DD1            0.0183      0.014      1.295      0.196        -0.009     0.046
DD2           -0.0007      0.012     -0.058      0.954        -0.025     0.024
DG1            0.0507      0.033      1.522      0.128        -0.015     0.116
DG2            0.0326      0.029      1.122      0.262        -0.024     0.090
==============================================================================
Omnibus:                      826.439   Durbin-Watson:                   2.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.050
Skew:                           0.034   Prob(JB):                     2.72e-10
Kurtosis:                       1.728   Cond. No.                         6.70
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### calculate  and inspect Variance Inflation Factor (VIF)

- Not correlated: $VIF=1$
- Moderately correlted: **$1<VIF<5$** or at a more conservative level of **$1<VIF <2.5$**
- Highly correlated: **$VIF>=5$** or at a more conservative level **$VIF>=5$**

#### $Win = \beta_{0} + \beta_{1}MeanC + \beta_{2}MeanW + \beta_{3}MeanD  \beta_{4}MeanG + e_{s}$

In [20]:
# For each X1, calculate VIF and save in dataframe
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif1['features'] = X1.columns
vif1.round(3)

,VIF Factor,features
0,61.903,const
1,1.148,MeanC
2,1.149,MeanW
3,1.248,MeanD
4,1.062,MeanG


the variance inflation factors for the constant of this model is **not acceptable** as $VIF<2.5$

In [21]:
# For each X2, calculate VIF and save in dataframe
vif2 = pd.DataFrame()
vif2['VIF Factor'] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif2['features'] = X2.columns
vif2.round(3)

,VIF Factor,features
0,2.636,const
1,1.148,meanc
2,1.149,meanw
3,1.248,meand
4,1.062,meang


the variance inflation factors for this model are **acceptable** as $VIF<2.5$

In [22]:
# For each X3, calculate VIF and save in dataframe
vif3 = pd.DataFrame()
vif3['VIF Factor'] = [variance_inflation_factor(X3.values, i) for i in range(X3.shape[1])]
vif3['features'] = X3.columns
vif3.round(3)

,VIF Factor,features
0,1.656,meanc
1,1.663,meanw
2,1.366,meand
3,1.112,meang


In [24]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/analysis/vif_2010-2017.tex', 'w')
f.write(beginningtex)
f.write(vif3.to_latex())
f.write(endtex)
f.close()

the variance inflation factors for this model are **acceptable** as $VIF<2.5$

In [25]:
# For each X4, calculate VIF and save in dataframe
vif4 = pd.DataFrame()
vif4['VIF Factor'] = [variance_inflation_factor(X4.values, i) for i in range(X4.shape[1])]
vif4['features'] = X4.columns
vif4.round(3)

,VIF Factor,features
0,23.794,MeanC
1,32.896,MeanW
2,37.938,MeanD
3,8.911,MeanG


the variance inflation factors for this model are **not acceptable** as $VIF > 2.5$

In [26]:
# For each X5, calculate VIF and save in dataframe
vif5 = pd.DataFrame()
vif5['VIF Factor'] = [variance_inflation_factor(X5.values, i) for i in range(X5.shape[1])]
vif5['features'] = X5.columns
vif5.round(3)

,VIF Factor,features
0,1.671,DC1
1,1.763,DC2
2,1.333,DC3
3,1.869,DW1
4,2.022,DW2
5,1.415,DW3
6,1.852,DD1
7,1.893,DD2
8,1.201,DG1
9,1.312,DG2


the variance inflation factors for DC2, DW2, DW3, DD1 in this model are **not acceptable** as $VIF > 2.5$ 

In [27]:
# For each X6, calculate VIF and save in dataframe
vif6 = pd.DataFrame()
vif6['VIF Factor'] = [variance_inflation_factor(X6.values, i) for i in range(X6.shape[1])]
vif6['features'] = X6.columns
vif6.round(3)

,VIF Factor,features
0,4.710,const
1,1.238,DC1
2,1.071,DC2
3,1.188,DC3
4,1.264,DW1
5,1.107,DW2
6,1.131,DW3
7,1.678,DD1
8,1.514,DD2
9,1.146,DG1


the variance inflation factors for the constant in this model is **not acceptable** as $VIF > 2.5$

## games with 12 forwards and 6 defensemen

In [37]:
dd = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2010_games_with_12f_6d.csv')
#dd = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2010_games_with_12f_6d.csv')
dd = dd.drop('Unnamed: 0', axis=1)

In [38]:
de = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2011_2017_games_with_12f_6d.csv')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2011_2017_games_with_12f_6d.csv')
de = de.drop('Unnamed: 0', axis=1)

In [39]:
dd.shape

(2460, 33)

In [40]:
de.shape

(14558, 33)

### join all seasons

In [41]:
dataframes = [dd, de]
df = pd.concat(dataframes)
df.head()

,Season,GameNumber,WinTeam,LossTeam,GF,GA,GD,TeamCode,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG,Sum
0,2010,20001,TOR,MTL,2,3,1,MTL,19.0,7.0,5.0,6.0,1.0,1.0,2.0,2.0,2.0,2.0,4.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,0,2.714286,2.000000,1.666667,1.0,7.380952
1,2010,20001,TOR,MTL,3,2,1,TOR,19.0,5.0,7.0,6.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,1,2.600000,2.285714,2.000000,3.0,9.885714
2,2010,20002,PHI,PIT,2,3,-1,PIT,19.0,8.0,4.0,6.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0,2.125000,2.000000,2.000000,2.0,8.125000
3,2010,20002,PHI,PIT,3,2,-1,PHI,19.0,5.0,7.0,6.0,1.0,3.0,0.0,2.0,0.0,3.0,1.0,2.0,0.0,1.0,0.0,3.0,2.0,0.0,2.0,1,1.800000,2.142857,1.833333,2.0,7.776190
4,2010,20003,CAR,MIN,3,4,-1,MIN,19.0,4.0,8.0,6.0,1.0,1.0,2.0,1.0,0.0,1.0,1.0,4.0,0.0,1.0,0.0,1.0,2.0,3.0,2.0,0,2.000000,2.750000,2.500000,2.0,9.250000


In [42]:
df.isnull().sum()

Season         0
GameNumber     0
WinTeam        0
LossTeam       0
GF             0
GA             0
GD             0
TeamCode       0
RosterCount    0
CCount         0
WCount         0
DCount         0
GCount         0
C1             0
C2             0
C3             0
C4             0
D1             0
D2             0
D3             0
G1             0
G2             0
G3             0
W1             0
W2             0
W3             0
W4             0
Win            0
MeanC          0
MeanW          0
MeanD          0
MeanG          0
Sum            0
dtype: int64

In [43]:
df.shape

(17018, 33)

In [44]:
df.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2010_2017_games_with_12f_6d.csv', index='False', sep=',')
#df.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2010_2017_games_with_12f_6d.csv', index='False', sep=',')

- summary analysis

In [45]:
df.groupby(['Win'])['MeanC', 'MeanW', 'MeanD', 'MeanG'].describe()

MeanC        MeanW        MeanD        MeanG
Win                                                          
0   count  8509.000000  8509.000000  8509.000000  8509.000000
    mean      2.186704     2.153370     1.935376     1.869139
    std       0.439123     0.363634     0.298114     0.665405
    min       1.000000     1.000000     1.166667     1.000000
    25%       1.875000     1.875000     1.666667     1.000000
    50%       2.200000     2.142857     1.833333     2.000000
    75%       2.500000     2.375000     2.166667     2.000000
    max       4.000000     3.666667     3.000000     3.000000
1   count  8509.000000  8509.000000  8509.000000  8509.000000
    mean      2.106883     2.074891     1.873703     1.714244
    std       0.423490     0.361078     0.279700     0.638626
    min       1.000000     1.000000     1.166667     1.000000
    25%       1.800000     1.833333     1.666667     1.000000
    50%       2.000000     2.000000     1.833333     2.000000
    75%       2.400000     2.285714     2.000000     2.000000
    max       4.000000     4.000000     3.000000     3.000000

### model estimation

- regress **team win percent** on the mean of players by position for games with 4 centers, 8 wingers and 6 defensemen.

In [46]:
df['meanc'] = 2.5 - df['MeanC']
df['meanw'] = 2.5 - df['MeanW']
df['meand'] = 2 - df['MeanD']
df['meang'] = 2 - df['MeanG']

df['DC1'] = df['C1'] - df['C4']
df['DC2'] = df['C2'] - df['C4']
df['DC3'] = df['C3'] - df['C4']

df['DW1'] = df['W1'] - df['W4']
df['DW2'] = df['W2'] - df['W4']
df['DW3'] = df['W3'] - df['W4']

df['DD1'] = df['D1'] - df['D3']
df['DD2'] = df['D2'] - df['D3']

df['DG1'] = df['G1'] - df['G3']
df['DG2'] = df['G2'] - df['G3']

In [47]:
w = df['Win'] 

Z1 = sm.add_constant(df[['MeanC', 'MeanW', 'MeanD', 'MeanG']] )
Z2 = sm.add_constant(df[['meanc', 'meanw', 'meand', 'meang']] )

Z3 = df[['meanc', 'meanw', 'meand', 'meang']]
Z4 = df[['MeanC', 'MeanW', 'MeanD', 'MeanG']]


n1 = sm.OLS(w, Z1).fit()
n2 = sm.OLS(w, Z2).fit()
n3 = sm.OLS(w, Z3).fit()
n4 = sm.OLS(w, Z4).fit()


#n1.summary()
#n2.summary()
n3.summary()
#n4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     2530.
Date:                Thu, 10 May 2018   Prob (F-statistic):               0.00
Time:                        18:44:13   Log-Likelihood:                -14278.
No. Observations:               17018   AIC:                         2.856e+04
Df Residuals:                   17014   BIC:                         2.860e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
meanc          0.3151      0.010     32.840      0.000         0.296     0.334
meanw          0.5241      0.010     50.225      0.000         0.504     0.545
meand          0.0006      0.016      0.038      0.970        -0.031     0.032
meang          0.0917      0.007     13.671      0.000         0.079     0.105
==============================================================================
Omnibus:                     6703.071   Durbin-Watson:                   2.254
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              847.749
Skew:                           0.004   Prob(JB):                    8.20e-185
Kurtosis:                       1.907   Cond. No.                         3.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
Z5 = df[['DC1', 'DC2', 'DC3', 'DW1', 'DW2', 'DW3', 'DD1', 'DD2', 'DG1', 'DG2']]
Z6 =  sm.add_constant(df[['DC1', 'DC2', 'DC3', 'DW1', 'DW2', 'DW3', 'DD1', 'DD2', 'DG1', 'DG2']])

n5 = sm.OLS(w, Z5).fit()
n6 = sm.OLS(w, Z6).fit()

n5.summary()
#n6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.425
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     1259.
Date:                Thu, 10 May 2018   Prob (F-statistic):               0.00
Time:                        18:44:20   Log-Likelihood:                -13535.
No. Observations:               17018   AIC:                         2.709e+04
Df Residuals:                   17008   BIC:                         2.717e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
DC1            0.0577      0.003     17.681      0.000         0.051     0.064
DC2            0.0545      0.003     21.084      0.000         0.049     0.060
DC3            0.0059      0.003      1.871      0.061        -0.000     0.012
DW1            0.0666      0.003     24.251      0.000         0.061     0.072
DW2            0.0645      0.002     29.782      0.000         0.060     0.069
DW3           -0.0018      0.003     -0.588      0.556        -0.008     0.004
DD1            0.0060      0.003      2.062      0.039         0.000     0.012
DD2            0.0152      0.002      6.682      0.000         0.011     0.020
DG1            0.0752      0.006     12.397      0.000         0.063     0.087
DG2            0.0575      0.005     10.504      0.000         0.047     0.068
==============================================================================
Omnibus:                    94950.354   Durbin-Watson:                   2.429
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1221.611
Skew:                           0.000   Prob(JB):                    5.38e-266
Kurtosis:                       1.687   Cond. No.                         5.92
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### calculate  and inspect Variance Inflation Factor (VIF)

- Not correlated: $VIF=1$
- Moderately correlted: **$1<VIF<5$** or at a more conservative level of **$1<VIF <2.5$**
- Highly correlated: **$VIF>=5$** or at a more conservative level **$VIF>=5$**

In [49]:
# For each Z1, calculate VIF and save in dataframe
vif11 = pd.DataFrame()
vif11['VIF Factor'] = [variance_inflation_factor(Z1.values, i) for i in range(Z1.shape[1])]
vif11['features'] = Z1.columns
vif11.round(3)

,VIF Factor,features
0,68.074,const
1,1.108,MeanC
2,1.149,MeanW
3,1.183,MeanD
4,1.056,MeanG


the variance inflation factors for the constant in this model is **not acceptable** as $VIF > 2.5$. 

In [50]:
# For each Z2, calculate VIF and save in dataframe
vif12 = pd.DataFrame()
vif12['VIF Factor'] = [variance_inflation_factor(Z2.values, i) for i in range(Z2.shape[1])]
vif12['features'] = Z2.columns
vif12.round(3)

,VIF Factor,features
0,2.521,const
1,1.108,meanc
2,1.149,meanw
3,1.183,meand
4,1.056,meang


the variance inflation factors for the constant in this model is **not acceptable** as $VIF > 2.5$

In [51]:
# For each Z3, calculate VIF and save in dataframe
vif13 = pd.DataFrame()
vif13['VIF Factor'] = [variance_inflation_factor(Z3.values, i) for i in range(Z3.shape[1])]
vif13['features'] = Z3.columns
vif13.round(3)

,VIF Factor,features
0,1.561,meanc
1,1.665,meanw
2,1.298,meand
3,1.159,meang


the variance inflation factors for this model are **acceptable** as $VIF > 2.5$

In [52]:
# For each Z4, calculate VIF and save in dataframe
vif14 = pd.DataFrame()
vif14['VIF Factor'] = [variance_inflation_factor(Z4.values, i) for i in range(Z4.shape[1])]
vif14['features'] = Z4.columns
vif14.round(3)

,VIF Factor,features
0,25.281,MeanC
1,33.012,MeanW
2,39.136,MeanD
3,8.807,MeanG


the variance inflation factors for all variables in this model are **not acceptable** as $VIF > 2.5$

In [53]:
# For each Z5, calculate VIF and save in dataframe
vif15 = pd.DataFrame()
vif15['VIF Factor'] = [variance_inflation_factor(Z5.values, i) for i in range(Z5.shape[1])]
vif15['features'] = Z5.columns
vif15.round(3)

,VIF Factor,features
0,1.780,DC1
1,1.577,DC2
2,1.296,DC3
3,1.975,DW1
4,1.775,DW2
5,1.414,DW3
6,1.711,DD1
7,1.763,DD2
8,1.190,DG1
9,1.262,DG2


the variance inflation factors for all variables in this model is **are acceptable** as $VIF > 2.5$

In [54]:
# For each Z6, calculate VIF and save in dataframe
vif16 = pd.DataFrame()
vif16['VIF Factor'] = [variance_inflation_factor(Z6.values, i) for i in range(Z6.shape[1])]
vif16['features'] = Z6.columns
vif16.round(3)

,VIF Factor,features
0,4.361,const
1,1.258,DC1
2,1.044,DC2
3,1.132,DC3
4,1.264,DW1
5,1.080,DW2
6,1.144,DW3
7,1.552,DD1
8,1.348,DD2
9,1.099,DG1


the variance inflation factors for the constant in this model is **not acceptable** as $VIF > 2.5$